# Experiment 03e: Attention Probing — Condition Examples

This notebook shows the actual text for each experimental condition using real data from the dataset. No GPU needed.

In [ ]:
import os, sys, json, re
import numpy as np
from pathlib import Path
from collections import Counter

sys.path.insert(0, ".")

SEED = 42
N_SAMPLES = 500

from datasets import load_dataset
ds = load_dataset("neural-bridge/rag-dataset-12000", split="train")

all_candidates = []
for row in ds:
    q = row.get("question", "")
    doc = row.get("context", "")
    answer = row.get("answer", "")
    if not q or not doc or not answer:
        continue
    q_words = len(q.split())
    a_words = len(answer.split())
    if q_words >= 15 and a_words >= 5:
        all_candidates.append({
            "query": q, "document": doc, "answer": answer,
            "query_words": q_words, "doc_words": len(doc.split()),
            "answer_words": a_words,
        })

np.random.seed(SEED)
indices = np.random.permutation(len(all_candidates))
samples = [all_candidates[i] for i in indices[:N_SAMPLES]]
del ds

# Verify against checkpoint
def verify_checkpoint(exp_name):
    ckpt_path = Path(f"results/{exp_name}/checkpoint.json")
    if ckpt_path.exists():
        ckpt = json.loads(ckpt_path.read_text())
        meta = ckpt.get('sample_meta', ckpt.get('results', []))
        if meta and meta[0].get('query', '')[:50] == samples[0]['query'][:50]:
            print(f"  Checkpoint verification: MATCH ({exp_name})")
            return True
    return None

print(f"Loaded {len(samples)} neural-bridge samples (SEED={SEED})")
print(f"Sample 0 query ({samples[0]['query_words']}w): {samples[0]['query'][:70]}")


def show_sample(s, doc_key='passage', n=0):
    # Show sample info
    doc = s[doc_key]
    print(f"{'='*80}")
    print(f"SAMPLE {n}")
    print(f"{'='*80}")
    print(f"  Query:    {s['query']}")
    print(f"  Answer:   {s['answer']}")
    print(f"  Document: {doc[:100]}...")
    print(f"  Doc words: {len(doc.split())}")
    print()

def show_conditions(conditions, doc_text):
    # conditions: list of (name, description, encoder_prefix_text_or_None)
    # For bare conditions, encoder_prefix_text is None
    print(f"{'Condition':<30} {'Prefix':<14} {'Encoder input (first 70 chars)'}")
    print(f"{'-'*100}")
    for name, desc, prefix_text in conditions:
        if prefix_text is None:
            enc_preview = doc_text[:70]
            print(f"{name:<30} {'(none)':<14} {enc_preview}...")
        else:
            enc_text = prefix_text + "\n" + doc_text
            print(f"{name:<30} {str(len(prefix_text.split()))+'w':<14} {enc_text[:70]}...")
        if desc:
            print(f"  {'':>28} ^ {desc}")
    print()

show_sample(samples[0], doc_key='document')
verify_checkpoint("exp03e")

ex = samples[0]
query_words = ex['query'].split()
n_qw = len(query_words)

other_idx = (0 + N_SAMPLES // 2) % N_SAMPLES
other_doc = samples[other_idx]['document']
rand_matched = " ".join(other_doc.split()[:n_qw])
repeat_the = " ".join(["the"] * n_qw)

conditions = [
    ("bare", "Document only — baseline encoder attention", None),
    ("oracle_trunc", "Real query — do prefix tokens get special attention?", ex['query']),
    ("random_matched_trunc", f"Random words, {n_qw}w — structural control", rand_matched),
    ("repeat_the_trunc", f"'the' x {n_qw} — minimal diversity control", repeat_the),
]

show_conditions(conditions, ex['document'])

print("PROBES (all measured at encoder attention layers):")
print("  A: Attention mass — how much doc-token attention goes to prefix?")
print("  B: Entropy — does prefix increase or decrease attention entropy?")
print("  C: Doc-doc redistribution — KL divergence of doc-doc attention pattern")
print("  D: Shift magnitude — L2 distance of doc representations (bare vs prefixed)")
print("  E: Shift direction — cosine similarity of shift vectors across conditions")
print("  F: Attention sinks — does prefix take over the sink role from <bos>?")
